In [9]:
import matplotlib
import operator
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib

# Set 
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = 10

# import custom functions
from functions_statistical_performance import *

In [10]:
# True to exactly reproduce the results presented in the paper (same classifiers)
reproduceflag=False

In [11]:
# function to predict over a generic db when you already have the classifiers
# it provides the average FPro 
def ClassifyDB(db, model_per_fold,nut_sel):
    dbsel=db.loc[:,nut_sel]
    Xnut=dbsel.values    
    
    indfold=0
    for model in model_per_fold:
        indfold+=1
        y_pred = model.predict(Xnut)
        y_probs= model.predict_proba(Xnut)
        db['classf'+str(indfold)]=y_pred
        db['p1f'+str(indfold)]=y_probs[:,0]
        db['p2f'+str(indfold)]=y_probs[:,1]
        db['p3f'+str(indfold)]=y_probs[:,2]
        db['p4f'+str(indfold)]=y_probs[:,3]
        db['FProf'+str(indfold)]=(1-db['p1f'+str(indfold)]+db['p4f'+str(indfold)])/2
     
    for p in range(1,5):
        db['p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].mean(axis=1)
        db['std_p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].std(axis=1)


    db['FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5' ]].mean(axis=1)
    db['std_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].std(axis=1)
    db['min_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].min(axis=1)


    db['max_p']=db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)
    db['class']=[int(s[1])-1 for s in db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)]
    db['min_in_which_fold']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)
    db['min_fold_id']=[int(s[-1]) for s in db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)]
    db['min_class']=[db['classf'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    
    for ind in range(1,5):
        db['p'+str(ind)+'_minFPro']=[db['p'+str(ind)+'f'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    

    return db

# Load 2009-2010 Data

In [12]:
# log-transformed nutrient content per 100 grams
RFFNDDS=pd.read_csv('./input_data/FNDDS_2009-10_Training_Data.csv')
RFFNDDS

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,macroclass,pythonlabel
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,-20.0,-20.0,-20.0,2047,"Salt, table",0,addition1516,2,Other,1
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,-20.0,-20.0,-20.0,11000000,"Milk, human",9602,Human milk,1,Milk,0
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,-20.0,-20.0,-20.0,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,Milk,0
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,-20.0,-20.0,-20.0,11111000,"Milk, cow's, fluid, whole",1002,"Milk, whole",1,Milk,0
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,-20.0,-20.0,-20.0,11111100,"Milk, cow's, fluid, whole, low-sodium",1002,"Milk, whole",1,Milk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94100200,"Water, bottled, sweetened, with low or no calo...",7802,Flavored or carbonated water,4,Other,3
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94100300,"Water, fruit flavored, sweetened, with high fr...",7802,Flavored or carbonated water,4,Other,3
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94210100,Propel Water,7804,Enhanced or fortified water,4,Other,3
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94210200,Glaceau Water,7804,Enhanced or fortified water,4,Other,3


In [13]:
# total nutrient list
nl=list(RFFNDDS.columns)
nl=nl[0:-7]
len(nl)

99

# Whole Nutrient Panel Analysis

In [14]:
if reproduceflag:
    model_per_foldWNP=joblib.load("./paper_classifiers_outcomes/FNDDS0910_99Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDS[RFFNDDS.pythonlabel!=-1]
    X=data2train.loc[:, 'Protein': 'Total isoflavones']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUCWNP, performancesAUPWNP, splitsWNP, model_per_foldWNP)=AUCAUPkfold(X,y,smoteflag)


AUC: average over the folds
[0.97708987 0.98032894 0.9694016  0.97989345]
AUC: std over the folds
[0.00310677 0.01303093 0.0093608  0.00228786]
AUP: average over the folds
[0.87484205 0.78024056 0.88138186 0.99155738]
AUP: std over the folds
[0.02447798 0.1044471  0.02887747 0.00098639]
Balanced Accuracy: average over the folds
0.7833123389310701
Balanced Accuracy: std over the folds
0.0373485452609236


In [15]:
# summary database
df=RFFNDDS.copy()
df=ClassifyDB(df,model_per_foldWNP, nl)
df.to_csv('./output_data/FNDDS_2009-10_99_Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.631250,p2,1,FProf5,5,1,0.020,0.677500,0.020,0.282500
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.292500,p1,0,FProf2,2,0,0.685,0.000000,0.045,0.270000
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.015000,p1,0,FProf5,5,0,0.985,0.000000,0.000,0.015000
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.015000,p1,0,FProf5,5,0,0.985,0.000000,0.000,0.015000
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.130000,p1,0,FProf2,2,0,0.855,0.015000,0.015,0.115000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.697083,p4,3,FProf5,5,3,0.265,0.070833,0.005,0.659167
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.890000,p4,3,FProf2,2,3,0.075,0.010000,0.060,0.855000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.957500,p4,3,FProf4,4,3,0.025,0.010000,0.025,0.940000
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.807708,p4,3,FProf2,2,3,0.120,0.054583,0.090,0.735417


# Standard Release FNDDS (for releases without flavonoids)

In [16]:
RFFNDDSSR=RFFNDDS.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
StandardNutPanel=list(RFFNDDSSR.columns)
RFFNDDSSR['pythonlabel']=RFFNDDS['pythonlabel'].copy()
print('Number of Nutrients in the Standard Nutrient Panel')
print(len(StandardNutPanel))

Number of Nutrients in the Standard Nutrient Panel
62


In [17]:
if reproduceflag:
    model_per_fold62nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_62Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSSR[RFFNDDSSR.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUC62nut, performancesAUP62nut, splits62nut, model_per_fold62nut)=AUCAUPkfold(X,y,smoteflag)

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.51033477 0.40919793 0.46776255 0.48549994]
AUC: std over the folds
[0.04045494 0.09218755 0.00941777 0.0072477 ]
AUP: average over the folds
[0.13092874 0.01658924 0.15086745 0.70480746]
AUP: std over the folds
[0.01159315 0.00234727 0.00351708 0.00579954]
Balanced Accuracy: average over the folds
0.24976369764266854
Balanced Accuracy: std over the folds
0.003098057761232834


In [18]:
# summary database
df=RFFNDDSSR.copy()
df=ClassifyDB(df,model_per_fold62nut,StandardNutPanel)
df.to_csv('./output_data/FNDDS_2009-10_62Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.753336,p4,3,FProf5,5,3,0.125387,0.023333,0.219220,0.632060
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.724375,p4,3,FProf3,3,3,0.150000,0.031250,0.220000,0.598750
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.633542,p3,2,FProf5,5,2,0.015000,0.006667,0.696250,0.282083
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.650417,p4,3,FProf5,5,2,0.005000,0.004167,0.685000,0.305833
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.853792,p4,3,FProf5,5,3,0.040000,0.019167,0.193250,0.747583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.727482,p4,3,FProf3,3,3,0.103905,0.043333,0.293893,0.558869
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.863750,p4,3,FProf4,4,3,0.052500,0.027500,0.140000,0.780000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.591667,p3,2,FProf2,2,2,0.053333,0.005000,0.705000,0.236667
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.858750,p4,3,FProf1,1,3,0.077500,0.025000,0.102500,0.795000


# Classify FNDDS 2015-2016 with 62 Nutrients

In [19]:
# log-transformed nutrient content per 100 grams
RFFNDDS20152016=pd.read_csv('./input_data/FNDDS_2015-16.csv')
df=ClassifyDB(RFFNDDS20152016,model_per_fold62nut,StandardNutPanel)
df.to_csv("./output_data/FNDDS_2015-16_62Nutrients_ensemble_5folds.csv")
df

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Food code,Main food description,year,cat_digit_1,cat_digit_2,cat_digit_3,cat_digit_4,cat_digit_5,WWEIA Category number,WWEIA Category description,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,11000000,"Milk, human",2015,Milk and Milk Products,Milks and milk drinks,"Milk, human",NaN,NaN,9602,Human milk,...,0.724375,p4,3,FProf3,3,3,0.150000,0.031250,0.220000,0.598750
1,11100000,"Milk, NFS",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1004,"Milk, reduced fat",...,0.699167,p4,3,FProf5,5,2,0.030000,0.006667,0.535000,0.428333
2,11111000,"Milk, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.650417,p4,3,FProf5,5,2,0.005000,0.004167,0.685000,0.305833
3,11111100,"Milk, low sodium, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.853792,p4,3,FProf5,5,3,0.040000,0.019167,0.193250,0.747583
4,11111150,"Milk, calcium fortified, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.650417,p4,3,FProf5,5,2,0.005000,0.004167,0.685000,0.305833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,99991400,Cheese as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.688958,p4,3,FProf2,2,3,0.150000,0.003750,0.318333,0.527917
8686,99995000,Breading or batter as ingredient in food,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.736208,p4,3,FProf4,4,3,0.140000,0.015000,0.232583,0.612417
8687,99995130,Wheat bread as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.673417,p4,3,FProf4,4,3,0.250333,0.040000,0.112500,0.597167
8688,99998130,Sauce as ingredient in hamburgers,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.557042,p4,3,FProf2,2,3,0.345917,0.020000,0.174083,0.460000


# Nutrition Facts Analysis

In [20]:
nutrition_facts=['Protein','Total Fat','Carbohydrate','Sugars, total','Fiber, total dietary','Calcium','Iron', 'Sodium','Retinol','Carotene, beta',
 'Carotene, alpha','Cryptoxanthin, beta','Vitamin C','Cholesterol', 'Fatty acids, total saturated'];
VitaminA=['Retinol','Carotene, beta','Carotene, alpha','Cryptoxanthin, beta']
RFFNDDSNF=RFFNDDS[nutrition_facts]
RFFNDDSNF['Total Vitamin A']=RFFNDDS[VitaminA].apply(np.exp).sum(axis=1).apply(np.log)
RFFNDDSNF=RFFNDDSNF.drop(VitaminA, axis=1)
nutrition_facts=list(RFFNDDSNF.columns)
print('Number of Nutrients in the Nutrition Facts Panel')
print(len(nutrition_facts))
RFFNDDSNF['pythonlabel']=RFFNDDS['pythonlabel'].copy()

Number of Nutrients in the Nutrition Facts Panel
12


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [21]:
if reproduceflag:
    model_per_fold12nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_12Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSNF[RFFNDDSNF.pythonlabel!=-1]
    X=data2train.loc[:, 'Protein': 'Total Vitamin A']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUC12nut, performancesAUP12nut, splits12nut, model_per_fold12nut)=AUCAUPkfold(X,y,smoteflag)

AUC: average over the folds
[0.98195239 0.98986574 0.96605625 0.97667158]
AUC: std over the folds
[0.00531323 0.0046968  0.00998409 0.00674639]
AUP: average over the folds
[0.89762263 0.77996147 0.85694722 0.99057707]
AUP: std over the folds
[0.01637637 0.10536019 0.02483225 0.00302229]
Balanced Accuracy: average over the folds
0.761806895095346
Balanced Accuracy: std over the folds
0.04202644174418826


In [22]:
# summary database
df=RFFNDDSNF.copy()
df=ClassifyDB(df,model_per_fold12nut,nutrition_facts)
df.to_csv('./output_data/FNDDS_2009-10_12Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,"Sugars, total","Fiber, total dietary",Calcium,Iron,Sodium,Vitamin C,Cholesterol,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.000000,-20.0,-3.729701,-8.016418,3.657337,-20.000000,-20.000000,...,0.594375,p2,1,FProf5,5,1,0.045000,0.6300,0.09125,0.233750
1,0.029559,1.477049,1.930071,1.930071,-20.0,-3.442019,-10.414313,-4.074542,-5.298317,-4.268698,...,0.330000,p1,0,FProf1,1,0,0.650000,0.0100,0.03000,0.310000
2,1.187843,0.647103,1.578979,1.625311,-20.0,-2.128632,-10.414313,-3.123566,-9.210340,-4.961845,...,0.000000,p1,0,FProf3,3,0,1.000000,0.0000,0.00000,0.000000
3,1.147402,1.178655,1.568616,1.619388,-20.0,-2.180367,-10.414313,-3.146555,-20.000000,-4.605170,...,0.015000,p1,0,FProf3,3,0,0.985000,0.0000,0.00000,0.015000
4,1.131402,1.241269,1.495149,1.495149,-20.0,-2.292635,-9.903488,-5.809143,-7.013116,-4.268698,...,0.169833,p1,0,FProf5,5,0,0.812667,0.0100,0.02500,0.152333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.000000,-20.0,-6.907755,-20.000000,-5.809143,-20.000000,-20.000000,...,0.320000,p4,3,FProf5,5,0,0.670000,0.0200,0.00000,0.310000
7250,-20.000000,-20.000000,1.504077,1.504077,-20.0,-4.074542,-20.000000,-4.828314,-4.366153,-20.000000,...,0.937500,p4,3,FProf2,2,3,0.055000,0.0050,0.01000,0.930000
7251,-20.000000,-20.000000,0.198851,0.198851,-20.0,-6.907755,-10.414313,-4.342806,-4.733004,-20.000000,...,0.955000,p4,3,FProf2,2,3,0.040000,0.0050,0.00500,0.950000
7252,-20.000000,-20.000000,1.702928,1.702928,-20.0,-4.074542,-20.000000,-20.000000,-4.366153,-20.000000,...,0.788750,p4,3,FProf1,1,3,0.145000,0.0125,0.12000,0.722500
